In [1]:
#Importing necessary libraries
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import pickle
import numpy as np


In [2]:
#Opening model
with open('finalized_model.pkl', 'rb') as f:
    model = pickle.load(f)

C:\Users\Jonathan\anaconda3\envs\recommendation-env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Gets API information
def get_keys(path):
    with open("/Users/Jonathan/Documents/Flatiron/phase_5/P5_spotify_recommendations/.secret/spotify_api.json") as f:
        return json.load(f)

In [4]:
#Gets spotify credentials
keys = get_keys("/Users/Jonathan/.secret/spotify_api.json")

client = keys['client']
api_key = keys['api_key']

In [5]:
#Authenticates using of API
auth_manager = SpotifyClientCredentials(client_id = client, client_secret = api_key)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [6]:
# Importing data
spot_df = pd.read_csv('data/spotify_playlist.csv')
spot_df.head()

,artist,artist_id,popularity,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Bon Jovi,58lV9VcRSjABbAbfWS6skp,8,Slippery When Wet,Livin' On A Prayer,0J6mQxEZnlRt9ymzFntA6z,0.534,0.887,0,-3.777,1,0.0345,0.000099,0.325,0.720,122.494,249293,4
1,Bon Jovi,58lV9VcRSjABbAbfWS6skp,0,Bon Jovi Greatest Hits,Wanted Dead Or Alive,4Zj9VM4fO1KwkU3lFaECsW,0.257,0.819,7,-3.562,1,0.0422,0.015000,0.324,0.267,150.354,308560,4
2,Queen,1dfeR4HaWDbWqFHLkxsg1d,64,News Of The World (Deluxe Remastered Version),We Are The Champions - Remastered 2011,7ccI9cStQbQdystvc6TvxD,0.268,0.459,7,-6.948,0,0.0346,0.000000,0.119,0.172,64.223,179200,4
3,Queen,1dfeR4HaWDbWqFHLkxsg1d,69,News Of The World (Deluxe Remastered Version),We Will Rock You - Remastered 2011,54flyrjcdnQdco7300avMJ,0.693,0.497,2,-7.316,1,0.1190,0.000000,0.258,0.473,81.308,122067,4
4,Bon Jovi,58lV9VcRSjABbAbfWS6skp,0,Cross Road,You Give Love A Bad Name,7LBJui5MdjfCd8YZr4xaqA,0.556,0.945,0,-4.347,0,0.0496,0.000004,0.394,0.781,122.875,224307,4


In [7]:

def get_song(song, artist):
    
    """
    Function
    ------------------------------------
    This function takes the song title and artist name and returns a Dataframe with a row of information that can be appended
    to an existing DataFrame
    
    """
    playlist_features_list = ["artist", "artist_id", "popularity", "album", "track_name", "track_id", 
                             "danceability", "energy", "key", "loudness", "mode", "speechiness",
                             "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"]

    song_df  = pd.DataFrame(columns = playlist_features_list)
    song = sp.search(q = 'track: {},  artist: {}'.format(song, artist), limit=1)
    for track in song:
    
        playlist_features = {}
        playlist_features["artist"] = song['tracks']['items'][0]['artists'][0]['name']
        playlist_features['artist_id'] = song['tracks']['items'][0]['artists'][0]['id']
        playlist_features['popularity'] = song['tracks']['items'][0]['popularity']
        playlist_features["album"] = song['tracks']['items'][0]['album']['name']
        playlist_features["track_name"] = song['tracks']['items'][0]['name']
        playlist_features["track_id"] = song['tracks']['items'][0]['id']

        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[6:]:
            playlist_features[feature] = audio_features[feature]
        
        track_df = pd.DataFrame(playlist_features, index = [0])
        song_df = pd.concat([song_df, track_df], ignore_index = True)
        return song_df

In [8]:
def predict(song_title, artist, df):
    
    """
    Function
    ------------------------------------
    This function gets the song outputted from the get_song function and concats it an imported DataFrame. It then runs 
    predictions to determine the clusters of each song and appends the cluster output to the DataFrame along with the new song.
    The function then filters the songs by the cluster of the song found as well as an x and y for each row.
    
    
    """
    song = get_song(song_title, artist)
    new_df = pd.concat([song, spot_df], ignore_index=True)
    new_df_feat = new_df.select_dtypes(np.number)
    preds = model.fit_predict(new_df_feat)
    coords = model.named_steps['umap'].transform(new_df_feat)
    new_df['cluster'] = preds
    new_df['x'], new_df['y'] = [x[0] for x in coords], [x[1] for x in coords]
    cluster = new_df.loc[new_df['track_name'] == new_df['track_name'][0], 'cluster'].to_list()[0]
    new_df = new_df.loc[new_df['cluster'] == cluster]
    
    
    return new_df

In [9]:
def dist(row):
    """
    Function
    ------------------------------------
    This function takes the x and y of a Dataframe and calculates the distance of each row
    
    """
    
    x = row['x']
    y = row['y']
    distance=np.sqrt((xt-x)**2 + (yt-y)**2)
    return distance
    

In [10]:
#Creating a variable to get the information to calculate the distance
new_df = predict('jimmy cooks', 'drake', spot_df)

C:\Users\Jonathan\anaconda3\envs\recommendation-env\lib\site-packages\sklearn\cluster\_kmeans.py:1848: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 4096 or by setting the environment variable OMP_NUM_THREADS=4
  warnings.warn(


In [11]:
#Calculates the distance of the given song and sort the values by the closest distance to the song found

xt = new_df.loc[new_df['track_name'] == new_df['track_name'][0],'x']
yt = new_df.loc[new_df['track_name'] == new_df['track_name'][0],'y']

new_df['distance'] = new_df.apply(dist, axis =1)
new_df.sort_values('distance')[0:10]
    

,artist,artist_id,popularity,album,track_name,track_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster,x,y,distance
0,Drake,3TVXtAsR1Inumwj472S9r4,91,"Honestly, Nevermind",Jimmy Cooks (feat. 21 Savage),3F5CgOj3wFlRv51JsHbxhe,0.529,0.673,0,-4.711,...,0.000002,0.0930,0.366,165.921,218365,4,3,9.058591,-1.995941,0.000000
6189,Rhett and Link,3XKtycltpy1gaeny0Bdzcs,20,Rhett & Link's Buddy System (Music from Season 2),Naked,4QafHRoPRbTy2iMvQ9n2nj,0.771,0.729,10,-4.964,...,0.000016,0.0535,0.935,149.928,96400,4,3,9.058790,-1.993983,0.001968
5262,2Pac,1ZwdS5xdxEREPySFridCfh,66,Greatest Hits,Hail Mary,2zoobJFEB9h15fjYjRd6oP,0.772,0.644,10,-6.318,...,0.000004,0.3660,0.388,82.050,312373,4,3,9.052939,-1.999187,0.006517
437,Cheap Trick,1LB8qB5BPb3MHQrfkvifXU,69,Heaven Tonight,Surrender,2ccUQnjjNWT0rsNnsBpsCA,0.524,0.955,0,-7.081,...,0.000009,0.6590,0.869,133.849,253733,4,3,9.048835,-1.990808,0.011024
3188,The Presidents Of The United States Of America,1lZvg4fNAqHoj6I9N8naBM,45,II,Volcano,09JxX9P3dqSl0I3lDGrTie,0.506,0.913,3,-5.301,...,0.000853,0.1350,0.894,151.182,178333,4,3,9.068841,-2.009458,0.016964
123,Fall Out Boy,4UXqAaa6dQYAk18Lv7PEgX,0,American Beauty/American Psycho,Uma Thurman,34Jeq8ktRzyOW1uw40InVy,0.624,0.953,4,-2.602,...,0.000128,0.6570,0.729,149.992,211573,4,3,9.041998,-1.985970,0.019359
2326,Seventh Day Slumber,6kFLnclYFc3gzpNt13wim5,4,A Decade Of Hope,Innocence,6EyVGwSrqHOk8OMJJ9rCu3,0.513,0.906,9,-5.948,...,0.000395,0.5920,0.429,81.512,213000,4,3,9.049955,-2.015767,0.021625
5255,Eminem,7dGJo4pcD2V6oG8kP0tJRR,73,The Eminem Show,Cleanin' Out My Closet,7BMO7O7ImjV8HNTH74Tshv,0.908,0.758,9,-4.753,...,0.000000,0.0783,0.870,148.015,297840,4,3,9.040853,-2.011480,0.023582
1605,Caribbean Crisis,469rxYEUIYNZnus1xiIfN2,0,Last Delight,Last Delight,1kiUIuYiKE0OMWK3TtVyRP,0.504,0.963,6,-5.417,...,0.039400,0.0527,0.631,159.922,247500,4,3,9.036553,-2.010020,0.026151
132,KISS,07XSN3sPlIlB2L2XNcTwJw,63,Crazy Nights,Crazy Crazy Nights,3YSfGJNDtF8KrHlwCAaVns,0.591,0.870,7,-6.381,...,0.000000,0.7180,0.747,121.682,227600,4,3,9.058474,-2.022204,0.026263
